In [1]:
import numpy as np
import os
home_path = os.path.expanduser("~") + '/'
from info_params import get_default_hparams
from utils.load_data import *
from sklearn.svm import SVC

OpenCV is built with OpenMP support. This usually results in poor performance. For details, see https://github.com/tensorpack/benchmarks/blob/master/ImageNet/benchmark-opencv-resize.py


In [2]:
def get_data(hps):
    dfX, df_next_deltaClose = load_data_seq(hps)

    segment, next_segment, target_one_hot = segment_seq(dfX, df_next_deltaClose, hps)

    train_segment, test_segment, _, _, train_target_one_hot, test_target_one_hot = \
        train_test_split(segment, next_segment, target_one_hot, hps)

    return train_segment, test_segment, train_target_one_hot, test_target_one_hot

In [3]:
hps = get_default_hparams()
hps.data_file_name = home_path + "data/cryptodatadownload/moving_average_240h.csv"
hps.attributes_normalize_mean = ['Close', 'Volume BTC', 'Spread High-Low', 'Spread Close-Open', "MA_Close","MA_V_BTC"]
hps.is_concat = True
hps.Tau = 1
hps.T = 30
hps.C = 2
hps.D = (1+hps.is_concat) * len(hps.attributes_normalize_mean)
hps.is_differencing = True
hps.lag_time = 1
hps.N_train_seq = 10000
hps.normalize_data = 'z_score'
hps.normalize_data_idx = True

In [4]:
X_train, X_test, train_target_one_hot, test_target_one_hot = get_data(hps)
X_train = np.reshape(X_train, newshape=[-1, hps.T * hps.D])
X_test = np.reshape(X_test, newshape=[-1, hps.T * hps.D])

y_train = np.argmax(train_target_one_hot, axis=-1)
y_train = np.reshape(y_train, newshape=[-1, hps.Tau])

y_test = np.argmax(test_target_one_hot, axis=-1)
y_test = np.reshape(y_test, newshape=[-1, hps.Tau])

Normalize: Z score


In [5]:
svclassifier = SVC(C=0.66, kernel='linear')
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_test)

/home/nam/miniconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Metrics in training data

In [6]:
from sklearn.metrics import classification_report, confusion_matrix
y_train_hat = svclassifier.predict(X_train)

cm = confusion_matrix(y_train, y_train_hat)
rp = classification_report(y_train, y_train_hat)

print(cm)
print(rp)

[[1170 3645]
 [ 755 4401]]
              precision    recall  f1-score   support

           0       0.61      0.24      0.35      4815
           1       0.55      0.85      0.67      5156

   micro avg       0.56      0.56      0.56      9971
   macro avg       0.58      0.55      0.51      9971
weighted avg       0.58      0.56      0.51      9971



# Metrics in validation data

In [7]:
y_pred = svclassifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
rp = classification_report(y_test, y_pred)

print(cm)
print(rp)

[[ 105 2233]
 [  95 2338]]
              precision    recall  f1-score   support

           0       0.53      0.04      0.08      2338
           1       0.51      0.96      0.67      2433

   micro avg       0.51      0.51      0.51      4771
   macro avg       0.52      0.50      0.38      4771
weighted avg       0.52      0.51      0.38      4771



# Save checkpoint

In [8]:
import os
import pickle

import os
f = os.getcwd() + "/checkpoint/svm-soft-margin.joblib"


my_file = open(f, 'wb')
my_file = pickle.dump(svclassifier, my_file)

# Load Checkpoint